Задача:

Выполнить парсинг двух страниц с сайта https://zakupki.gov.ru/ по каждой закупке спарсить номер закупки, дату размещения, обновления и окончание подачи заявок, начальную цену контракта. Данные сохранить в pandas датафрейм и в формат csv.

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
def get_tenders_from_page(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code != 200:
        print(f"Ошибка при получении страницы {url}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    tenders = []
    
    for row in soup.select('.search-registry-entry-block'):  # Класс может изменяться
        try:
            tender_number = row.select_one('.registry-entry__header-mid__number a')
            placement_date = row.select_one('.data-block__value')
            update_date_elements = row.select('.data-block__value')
            price = row.select_one('.price-block__value')
            
            if tender_number and placement_date and price and len(update_date_elements) >= 3:
                tenders.append({
                    "Номер закупки": tender_number.text.strip(),
                    "Дата размещения": placement_date.text.strip(),
                    "Дата обновления": update_date_elements[1].text.strip(),
                    "Окончание подачи заявок": update_date_elements[2].text.strip(),
                    "Начальная цена": price.text.strip()
                })
        except AttributeError:
            continue
    
    return tenders

In [13]:
# Задаем ссылки на две страницы
urls = [
    "https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on",
    "https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&page=2"
]

In [14]:
data = []
for url in urls:
    data.extend(get_tenders_from_page(url))

In [15]:
df = pd.DataFrame(data)

In [16]:
df.head(3)

,Номер закупки,Дата размещения,Дата обновления,Окончание подачи заявок,Начальная цена
0,№ 0138300016025000002,19.03.2025,22.03.2025,27.03.2025,"12 771 666,67 ₽"
1,№ 32514614418,14.03.2025,22.03.2025,28.03.2025,"8 023 333,33 ₽"
2,№ 0351300020025000077,22.03.2025,22.03.2025,31.03.2025,"62 407,10 ₽"


In [17]:
# Создаем DataFrame и сохраняем в CSV
if data:
    df = pd.DataFrame(data)
    df.to_csv("zakupki_data.csv", index=False, encoding="utf-8-sig")
    print("Данные успешно сохранены в zakupki_data.csv")
else:
    print("Не удалось получить данные.")

Данные успешно сохранены в zakupki_data.csv
